In [1]:
import gensim
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pandas as pd
train = pd.read_csv('./small_data/train.csv')
test = pd.read_csv('./small_data/test.csv')

In [4]:
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('~/datasets/GoogleNews-vectors-negative300.bin', binary=True)

In [15]:
from nltk import tokenize
import numpy as np

def sentence_to_vec(sentence, vectors):
    vec_size = 300
    tokenizer = tokenize.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    
#     sentence_vec = np.zeros(vec_size)
    sentences = []
    
    for token in tokens:
        if token in vectors:
            sentences.append(vectors[token])
    
    return np.array(sentences)

In [16]:
print train['str1'][0]
s1 = sentence_to_vec(train['str1'][0], word_vectors)
print train['str2'][0]
s2 = sentence_to_vec(train['str2'][0], word_vectors)

print

print train['str1'][1]
s3 = sentence_to_vec(train['str1'][1], word_vectors)
print train['str2'][1]
s4 = sentence_to_vec(train['str2'][1], word_vectors)

print

print train['str1'][2]
s5 = sentence_to_vec(train['str1'][2], word_vectors)
print train['str2'][2]
s6 = sentence_to_vec(train['str2'][2], word_vectors)


Amrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence.
Referring to him as only "the witness", Amrozi accused his brother of deliberately distorting his evidence.

Yucaipa owned Dominick's before selling the chain to Safeway in 1998 for $2.5 billion.
Yucaipa bought Dominick's in 1995 for $693 million and sold it to Safeway for $1.8 billion in 1998.

They had published an advertisement on the Internet on June 10, offering the cargo for sale, he added.
On June 10, the ship's owners had published an advertisement on the Internet, offering the explosives for sale.


In [17]:
s1.shape

(13, 300)

In [7]:
print cosine_similarity(s1, s2), train['paraphrase'][0]
print cosine_similarity(s3, s4), train['paraphrase'][1]
print cosine_similarity(s5, s6), train['paraphrase'][2]

[[ 0.97637547]] 1
[[ 0.91275351]] 0
[[ 0.91011708]] 1


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

In [8]:
def batch_sentences(sentences):
    v = [sentence_to_vec(x, word_vectors) for x in sentences]
    return np.array(v)

In [9]:
train_vecs1 = batch_sentences(train['str1'])
train_vecs2 = batch_sentences(train['str2'])

test_vecs1 = batch_sentences(test['str1'])
test_vecs2 = batch_sentences(test['str2'])

/usr/local/lib/python2.7/dist-packages/gensim/models/keyedvectors.py:585: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return word in self.vocab


In [32]:
with open('./small_data/train_word2vec_s1.npy', 'w') as f:
    np.save(f, train_vecs1)
    
with open('./small_data/train_word2vec_s2.npy', 'w') as f:
    np.save(f, train_vecs2)

with open('./small_data/test_word2vec_s1.npy', 'w') as f:
    np.save(f, test_vecs1)
    
with open('./small_data/test_word2vec_s2.npy', 'w') as f:
    np.save(f, test_vecs2)

In [33]:
def compute_cosine(v1, v2):
    distances = []
    for xi, xj in zip(v1, v2):
        distances.append(cosine_similarity(xi, xj))
    distances = map(lambda x: x[0], distances)
    dist_vec = np.array(distances)
    return dist_vec

In [34]:
# models on models on models, bottles on bottles on bottles

In [39]:
X_train = compute_cosine(train_vecs1, train_vecs2)
X_test = compute_cosine(test_vecs1, test_vecs2)

train_labels = np.load('./small_data/train_labels.npy')
test_labels = np.load('./small_data/test_labels.npy')

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

In [42]:
X_t = np.column_stack((X_train, train_vecs2, train_vecs1))
X_tst = np.column_stack((X_test, test_vecs1, test_vecs2))

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(verbose=True, n_estimators=50, max_depth=30, loss='exponential')
# clf = LogisticRegression(verbose=1, n_jobs=-1)
# clf = SVC(verbose=True)
clf.fit(X_t, train_labels)
print clf.score(X_tst, test_labels)

#0.72869565217391308
# n_estimators=300, max_depth=8 -> 0.725797101449

      Iter       Train Loss   Remaining Time 
         1           0.8516            1.26m
         2           0.7724            1.34m
         3           0.6995            1.35m
         4           0.6330            1.35m
         5           0.5732            1.33m
         6           0.5188            1.32m
         7           0.4697            1.29m
         8           0.4251            1.27m
         9           0.3847            1.24m
        10           0.3482            1.22m
        20           0.1288           54.44s
        30           0.0480           35.89s
        40           0.0183           17.63s
        50           0.0073            0.00s
0.649275362319


In [14]:
import keras
import keras.backend as K
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Embedding, Dropout

In [29]:
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

vocab_size = 10000
emb_dim = 300

def create_lstm_nn(input_dim):
    seq = Sequential()
    # encode via LSTM
    seq.add(LSTM(128, input_shape=input_dim))
    seq.add(Dropout(0.3))
    return seq

In [32]:
lstm1 = create_lstm_nn((300,20))
lstm2 = create_lstm_nn((300,20))